In [1]:
import k3d
import numpy as np

from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED

from scipy.spatial.transform import Rotation as R

In [2]:
my_mpmt = MPMT(None, 'M2', {}, {})

origins = []
vectors = []
n_mp = 20
meshes = []
radius = my_mpmt.pmts[0].prop_design['size']/2.

for i_pmt, pmt in enumerate(my_mpmt.pmts):
    location, direction = pmt.get_z_orientation('true')

    lv = [direction[i]*100 for i in range(len(direction))]
    origins.append(location)
    vectors.append(lv)

    mesh = []
    if i_pmt > -1:
        # draw the surface of the pmt with n_mp points
        # find a direction perpendicular to normal p = (px, 0., 1.)
        px = -direction[2]/(direction[0]+1.E-3)
        # change magnitude to PMT radius
        scale = radius/np.sqrt(1. + px**2)
        perp = (px*scale,0.,scale)

        rot = R.from_rotvec(2.*np.pi/n_mp * np.array(direction))
        for i in range(n_mp):
            mesh.append(np.add(location,perp))
            perp = rot.apply(perp)
        meshes.append(np.array(mesh,dtype=np.float32))


In [ ]:
# plot the normal vectors

plt_vectors = k3d.vectors(origins=origins, vectors=vectors)

plot = k3d.plot()
plot += plt_vectors
plot.display()

In [ ]:
# show the PMT faces with numbering

indices = []
for i in range(n_mp):
    indices.append([i,(i+1)%n_mp])


for i_mesh,mesh in enumerate(meshes):
    plt_mesh = k3d.lines(mesh, indices, indices_type='segment')
    plot += plt_mesh
    plt_text = k3d.text(str(i_mesh), position=origins[i_mesh], reference_point='cc', size=1., label_box=False)
    plot += plt_text

plot.display()